# Python 协程

## 什么是协程？

协程（Coroutine）字面理解协作完成任务，而非线程的抢占式多任务。

是实现并发编程的一种方式。一说并发，就让人想到了多线程 / 多进程模型，没错，多线程 / 多进程，正是解决并发问题的经典模型之一。最初的互联网世界，多线程 / 多进程在服务器并发中，起到举足轻重的作用。

事件循环（event loop）启动一个统一的调度器，让调度器来决定一个时刻去运行哪个任务，于是*省却了多线程中启动线程、管理线程、同步锁等各种开销*。让人惊喜地发现，这种工具完美地继承了事件循环的优越性，同时还能提供 async / await 语法糖，解决了执行性和可读性共存的难题。于是，协程逐渐被更多人发现并看好。

回到我们的 Python。使用生成器，是 Python 2 时代实现协程的老方法，Python 3.7 提供了新的基于 asyncio 和 async / await 的方法。我们直接来讲新方法。

## 从一个爬虫说起


In [1]:
import time

def crawl_page(url):
    print('crawling {}'.format(url))
    sleep_time = int(url.split('_')[-1])
    time.sleep(sleep_time)
    print('OK {}'.format(url))

def main(urls):
    for url in urls:
        crawl_page(url)

%time main(['url_1', 'url_2', 'url_3', 'url_4'])

crawling url_1
OK url_1
crawling url_2
OK url_2
crawling url_3
OK url_3
crawling url_4
OK url_4
CPU times: user 3.68 ms, sys: 2.02 ms, total: 5.71 ms
Wall time: 10 s


In [2]:
import time
import functools

def async_time_it(func):
    @functools.wraps(func)
    async def wrapper(*args, **kwargs):
        start = time.perf_counter()
        res = await func(*args, **kwargs)
        end = time.perf_counter()
        print(f'\nTook {(end - start):.3f} s')
        return res
    return wrapper

In [3]:
import asyncio

async def crawl_page(url):
    print('crawling {}'.format(url))
    sleep_time = int(url.split('_')[-1])
    await asyncio.sleep(sleep_time)
    print('OK {}'.format(url))

async def main(urls):
    for url in urls:
        await crawl_page(url)

@async_time_it
async def run():
    await main(['url_1', 'url_2', 'url_3', 'url_4'])

await run()

crawling url_1
OK url_1
crawling url_2
OK url_2
crawling url_3
OK url_3
crawling url_4
OK url_4

Took 10.008 s


## 协程的执行

执行协程有多种方法，这里我介绍一下常用的三种。

- 首先，我们可以通过 await 来调用。await 执行的效果，和 Python 正常执行是一样的，也就是说程序会阻塞在这里，进入被调用的协程函数，执行完毕返回后再继续，而这也是 await 的字面意思。

   代码中 await asyncio.sleep(sleep_time) 会在这里休息若干秒，await crawl_page(url) 则会执行 crawl_page() 函数。
   
- 其次，我们可以通过 asyncio.create_task() 来创建任务。
- 最后，我们需要 asyncio.run 来触发运行。asyncio.run 这个函数是 Python 3.7 之后才有的特性，可以让 Python 的协程接口变得非常简单，你不用去理会事件循环怎么定义和怎么使用的问题。一个非常好的编程规范是，asyncio.run(main()) 作为主程序的入口函数，在程序运行周期内，只调用一次 asyncio.run。

In [4]:
import asyncio

async def crawl_page(url):
    print('crawling {}'.format(url))
    sleep_time = int(url.split('_')[-1])
    await asyncio.sleep(sleep_time)
    print('OK {}'.format(url))

async def main(urls):
    tasks = [asyncio.create_task(crawl_page(url)) for url in urls]
    for task in tasks:
        await task

@async_time_it
async def run():
    await main(['url_1', 'url_2', 'url_3', 'url_4'])

await run()

crawling url_1
crawling url_2
crawling url_3
crawling url_4
OK url_1
OK url_2
OK url_3
OK url_4

Took 4.001 s


In [5]:
import asyncio

async def worker_1():
    print('worker_1 start')
    await asyncio.sleep(1)
    print('worker_1 done')

async def worker_2():
    print('worker_2 start')
    await asyncio.sleep(2)
    print('worker_2 done')

@async_time_it
async def main():
    print('before await')
    await worker_1()
    print('awaited worker_1')
    await worker_2()
    print('awaited worker_2')

await main()

before await
worker_1 start
worker_1 done
awaited worker_1
worker_2 start
worker_2 done
awaited worker_2

Took 3.002 s


In [6]:
import asyncio

async def worker_1():
    print('worker_1 start')
    await asyncio.sleep(1)
    print('worker_1 done')

async def worker_2():
    print('worker_2 start')
    await asyncio.sleep(2)
    print('worker_2 done')

@async_time_it
async def main():
    task1 = asyncio.create_task(worker_1())
    task2 = asyncio.create_task(worker_2())

    print('before await')
    await task1
    print('awaited worker_1')
    await task2
    print('awaited worker_2')

await main()

before await
worker_1 start
worker_2 start
worker_1 done
awaited worker_1
worker_2 done
awaited worker_2

Took 2.003 s


让我们来看一下上面的代码都做了些什么？

1. await main()，程序进入 main() 函数，事件循环开启；
1. task1 和 task2 任务被创建，并进入事件循环等待运行；
1. 运行到 print，输出 'before await'；
1. await task1 执行，用户选择从当前的主任务中切出，事件调度器开始调度 worker_1；
1. worker_1 开始运行，运行 print 输出'worker_1 start'，然后运行到 await asyncio.sleep(1)， 从当前任务切出，事件调度器开始调度 worker_2；
1. worker_2 开始运行，运行 print 输出 'worker_2 start'，然后运行 await asyncio.sleep(2) 从当前任务切出；

   以上所有事件的运行时间，都应该在 1ms 到 10ms 之间，甚至可能更短，事件调度器从这个时候开始暂停调度；

1. 一秒钟后，worker_1 的 sleep 完成，事件调度器将控制权重新传给 task_1，输出 'worker_1 done'，task_1 完成任务，从事件循环中退出；
1. await task1 完成，事件调度器将控制器传给主任务，输出 'awaited worker_1'，然后在 await task2 处继续等待；
1. 两秒钟后，worker_2 的 sleep 完成，事件调度器将控制权重新传给 task_2，输出 'worker_2 done'，task_2 完成任务，从事件循环中退出；
1. 主任务输出 'awaited worker_2'，协程全任务结束，事件循环结束。

如果我们想给某些协程任务限定运行时间，一旦超时就取消，又该怎么做呢？再进一步，如果某些协程运行时出现错误，又该怎么处理呢？


In [7]:
import asyncio

async def worker_1():
    await asyncio.sleep(1)
    return 1

async def worker_2():
    await asyncio.sleep(2)
    return 2 / 0

async def worker_3():
    await asyncio.sleep(3)
    return 3

@async_time_it
async def main():
    task_1 = asyncio.create_task(worker_1())
    task_2 = asyncio.create_task(worker_2())
    task_3 = asyncio.create_task(worker_3())

    # 这种写法比较特别：2 秒后取消 task_3。如果那时 task_3 已经完成，这个语句相当于无效语句
    await asyncio.sleep(2)
    task_3.cancel()

    # 很关键的 return_exceptions=True，避免直接抛出异常到主线程
    res = await asyncio.gather(task_1, task_2, task_3, return_exceptions=True)
    print(res)

await main()

[1, ZeroDivisionError('division by zero'), CancelledError()]

Took 2.002 s


注意 `return_exceptions=True` 这行代码。如果不设置这个参数，错误就会完整地 throw 到我们这个执行层，从而需要 try except 来捕捉，这也就意味着其他还没被执行的任务会被全部取消掉。

In [9]:
import asyncio
import random

async def consumer(queue, id):
    while True:
        val = await queue.get()
        print('<< {} get a val: {}'.format(id, val))
        await asyncio.sleep(1)

async def producer(queue, id):
    for i in range(5):
        val = random.randint(1, 10)
        await queue.put(val)
        print('>> {} put a val: {}'.format(id, val))
        await asyncio.sleep(1)

@async_time_it
async def main():
    queue = asyncio.Queue()

    consumer_1 = asyncio.create_task(consumer(queue, 'consumer_1'))

    producer_1 = asyncio.create_task(producer(queue, 'producer_1'))
    producer_2 = asyncio.create_task(producer(queue, 'producer_2'))

    # 10 秒后取消消费队列
    await asyncio.sleep(10)
    consumer_1.cancel()
    
    await asyncio.gather(consumer_1, producer_1, producer_2, return_exceptions=True)

await main()

>> producer_1 put a val: 7
>> producer_2 put a val: 8
<< consumer_1 get a val: 7
>> producer_1 put a val: 10
>> producer_2 put a val: 5
<< consumer_1 get a val: 8
>> producer_1 put a val: 7
>> producer_2 put a val: 4
<< consumer_1 get a val: 10
>> producer_1 put a val: 4
>> producer_2 put a val: 5
<< consumer_1 get a val: 5
>> producer_1 put a val: 2
>> producer_2 put a val: 5
<< consumer_1 get a val: 7
<< consumer_1 get a val: 4
<< consumer_1 get a val: 4
<< consumer_1 get a val: 5
<< consumer_1 get a val: 2
<< consumer_1 get a val: 5

Took 10.002 s


## 实战：豆瓣近日推荐电影爬虫

任务描述：

https://movie.douban.com/cinema/later/beijing/ 这个页面描述了北京最近上映的电影，通过 Python 得到这些电影的名称、上映时间和海报。


需要预安装 requests、bs4 和 lxml

```shell
pip3 install requests bs4 lxml aiohttp
```

>参考
>
>- [requests](https://requests.readthedocs.io/en/master/user/quickstart/)
>- [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc.zh/)
>- [aiohttp](https://docs.aiohttp.org/en/stable/)

- 同步版代码


In [10]:
import requests
from bs4 import BeautifulSoup

RES_URL = "https://movie.douban.com/cinema/later/beijing/"
HEADERS = {'user-agent': 'demo/0.9.0'}

# 为了方便使用 BeautifulSoup 分析，以下代码可以先存储 HTTP 请求结果
# res = requests.get(RES_URL, headers=HEADERS)
# init_page = res.content
# 存储
# %store init_page
# 加载
# %store -r init_page
# print(len(init_page))

def main():
    res = requests.get(RES_URL, headers=HEADERS)
    init_page = res.content
    init_soup = BeautifulSoup(init_page, 'lxml')
    all_movies = init_soup.find('div', id="showing-soon")
    for each_movie in all_movies.find_all('div', class_="item"):
        all_a_tag = each_movie.find_all('a')
        all_li_tag = each_movie.find_all('li')
        
        movie_name = all_a_tag[1].text
        url_to_fetch = all_a_tag[1]['href']
        movie_date = all_li_tag[0].text

        # all_img_tag = each_movie.find_all('img')
        # img_tag = all_img_tag[0]

        # 本可以直接使用上面代码获取图片，此处故意增加 HTTP 请求
        response_item = requests.get(url_to_fetch, headers=HEADERS).content
        soup_item = BeautifulSoup(response_item, 'lxml')
        img_tag = soup_item.find('img')

        print('{} {} {}'.format(movie_name, movie_date, img_tag['src']))

%time main()

应承 11月26日 https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2626077915.jpg
妈妈不再摇滚 11月26日 https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2615614077.jpg
一秒钟 11月27日 https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2623809595.jpg
疯狂原始人2 11月27日 https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2624607255.jpg
日光之下 11月27日 https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2625833086.jpg
闺蜜心窍 11月27日 https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2623348372.jpg
隐形人 12月04日 https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2624870548.jpg
赤狐书生 12月04日 https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2622753154.jpg
怪物猎人 12月04日 https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2625920629.jpg
如果声音不记得 12月04日 https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2557141890.jpg
宝可梦：超梦的逆袭 进化 12月04日 https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2625913008.jpg
五彩缤纷 12月04日 https://img9.doubani

- 异步版代码

In [11]:
import asyncio
import aiohttp

from bs4 import BeautifulSoup

RES_URL = "https://movie.douban.com/cinema/later/beijing/"
HEADERS = {'user-agent': 'demo/0.9.0'}
DEBUG = False


async def fetch_content(url, headers=HEADERS):
    async with aiohttp.ClientSession(
        headers=headers, connector=aiohttp.TCPConnector(ssl=False)
    ) as session:
        async with session.get(url) as response:
            if DEBUG:
                print("Status:", response.status)
                print("Content-type:", response.headers['content-type'])
            return await response.text()

@async_time_it
async def main():
    init_page = await fetch_content(RES_URL)
    init_soup = BeautifulSoup(init_page, 'lxml')

    movie_names, urls_to_fetch, movie_dates = [], [], []

    all_movies = init_soup.find('div', id="showing-soon")
    for each_movie in all_movies.find_all('div', class_="item"):
        all_a_tag = each_movie.find_all('a')
        all_li_tag = each_movie.find_all('li')

        movie_names.append(all_a_tag[1].text)
        urls_to_fetch.append(all_a_tag[1]['href'])
        movie_dates.append(all_li_tag[0].text)

    tasks = [fetch_content(url) for url in urls_to_fetch]
    pages = await asyncio.gather(*tasks)

    for movie_name, movie_date, page in zip(movie_names, movie_dates, pages):
        soup_item = BeautifulSoup(page, 'lxml')
        img_tag = soup_item.find('img')

        print('{} {} {}'.format(movie_name, movie_date, img_tag['src']))

await main()

应承 11月26日 https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2626077915.jpg
妈妈不再摇滚 11月26日 https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2615614077.jpg
一秒钟 11月27日 https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2623809595.jpg
疯狂原始人2 11月27日 https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2624607255.jpg
日光之下 11月27日 https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2625833086.jpg
闺蜜心窍 11月27日 https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2623348372.jpg
隐形人 12月04日 https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2624870548.jpg
赤狐书生 12月04日 https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2622753154.jpg
怪物猎人 12月04日 https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2625920629.jpg
如果声音不记得 12月04日 https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2557141890.jpg
宝可梦：超梦的逆袭 进化 12月04日 https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2625913008.jpg
五彩缤纷 12月04日 https://img9.doubani

## 总结

- 协程和多线程的区别，主要在于两点，一是协程为单线程；二是协程由用户决定，在哪些地方交出控制权，切换到下一个任务。
- 协程的写法更加简洁清晰，把 async / await 语法和 create_task 结合来用，对于中小级别的并发需求已经毫无压力。
- 写协程程序的时候，脑海中要有清晰的事件循环概念，知道程序在什么时候需要暂停、等待 I/O，什么时候需要一并执行到底。

## 思考题

协程怎么实现回调函数呢？

TBD